## A3c: Perform a Tobit regression analysis on "NSSO68.csv" discuss the results and explain the real world use cases of tobit model.

Name : Aakash Kathirvel

VID : V01110153

Date : 07/01/2024

In [3]:
pip install pandas numpy statsmodels

Note: you may need to restart the kernel to use updated packages.


In [46]:
#Import Required Libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression import quantile_regression
from sklearn.preprocessing import StandardScaler
import scipy.optimize as opt
from scipy.stats import norm

In [47]:
# Load the dataset
data = pd.read_csv("C:/Users/Aakash/Desktop/SCMA/NSSO68.csv")
data.head()

C:\Users\Aakash\AppData\Local\Temp\ipykernel_13848\2058581863.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:/Users/Aakash/Desktop/SCMA/NSSO68.csv")


,slno,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,...,pickle_v,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot
0,1,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2,12.000000,154.18
1,2,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2,333.000000,484.95
2,3,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2,35.000000,214.84
3,4,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2,168.333333,302.30
4,5,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2,15.000000,148.00


In [48]:
# Select relevant columns
selected_cols = ["MPCE_URP", "Age", "Sex", "Education", "Religion", "hhdsz"]
data_selected = data[selected_cols]

In [49]:
# Handle missing values
data_selected = data_selected.dropna()

In [50]:
# Convert categorical variables to dummy variables
data_selected = pd.get_dummies(data_selected, columns=['Sex', 'Religion', 'Education'], drop_first=True)

In [51]:
# Ensure all data is in numeric format
data_selected = data_selected.apply(pd.to_numeric, errors='coerce')

In [52]:
# Define censored data (left and right bounds)
censored = (data_selected["MPCE_URP"] == 0)  # Assuming censoring at 0

In [53]:
# Scale the data
scaler = StandardScaler()
data_selected_scaled = scaler.fit_transform(data_selected.drop(columns=["MPCE_URP"]))

In [54]:
# Define the Tobit likelihood function
def tobit_likelihood(params, y, X, censored):
    beta = params[:-2]
    sigma = params[-2]
    nu = params[-1]
    
    mu = np.dot(X, beta)
    resid = (y - mu) / sigma
    pdf = norm.pdf(resid)
    cdf = norm.cdf(nu)
    
    LL = np.sum(np.log(np.where(censored, cdf, pdf / sigma)))
    return -LL


In [55]:
# Initial guess for parameters
mean_y = np.mean(data_selected["MPCE_URP"])
initial_params = np.hstack((np.zeros(data_selected.shape[1]), [1.0, mean_y]))

# Exogenous variables
X = sm.add_constant(data_selected_scaled)

# Endogenous variable
y = data_selected["MPCE_URP"]

In [56]:

# Estimate parameters using maximum likelihood estimation (MLE)
results = opt.minimize(tobit_likelihood, initial_params, args=(y, X, censored), method='BFGS')

# Extract estimated parameters
beta_est = results.x[:-2]
sigma_est = results.x[-2]
nu_est = results.x[-1]


C:\Users\Aakash\AppData\Local\Temp\ipykernel_13848\4082158152.py:12: RuntimeWarning: divide by zero encountered in log
  LL = np.sum(np.log(np.where(censored, cdf, pdf / sigma)))
c:\Users\Aakash\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [57]:
# Print results
print("Coefficients:")
print(pd.DataFrame({"Variable": ["const"] + data_selected.drop(columns=["MPCE_URP"]).columns.tolist(), 
                    "Coefficient": beta_est}))

# Summary of the model
print("\nSigma (Standard deviation of errors):", sigma_est)
print("Nu (Location parameter):", nu_est)


Coefficients:
          Variable  Coefficient
0            const          0.0
1              Age          0.0
2            hhdsz          0.0
3            Sex_2          0.0
4     Religion_2.0          0.0
5     Religion_3.0          0.0
6     Religion_4.0          0.0
7     Religion_5.0          0.0
8     Religion_6.0          0.0
9     Religion_7.0          0.0
10    Religion_9.0          0.0
11   Education_2.0          0.0
12   Education_3.0          0.0
13   Education_4.0          0.0
14   Education_5.0          0.0
15   Education_6.0          0.0
16   Education_7.0          0.0
17   Education_8.0          0.0
18  Education_10.0          0.0
19  Education_11.0          0.0
20  Education_12.0          0.0
21  Education_13.0          0.0

Sigma (Standard deviation of errors): 1.0
Nu (Location parameter): 2050.83056762287
